In [ ]:
!pip install nltk
!pip install pronouncing


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
import random
import re

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import json
import os

# Read the poems from the provided JSON file
def read_poems_from_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        poems_data = json.load(f)

    all_poems = ""
    for poem in poems_data:
        all_poems += poem['text'].replace("<START>", "").replace("<END>", "") + "\n\n"
    return all_poems

# Prepare the input text sequences for the LSTM model
def prepare_sequences(text, max_sequence_len):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts([text])

    sequences = []
    for line in text.split('\n'):
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            sequences.append(n_gram_sequence)

    max_sequence_len = max([len(seq) for seq in sequences])
    sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='pre')

    X, y = sequences[:,:-1], sequences[:,-1]
    y = tf.keras.utils.to_categorical(y, num_classes=len(tokenizer.word_index) + 1)

    return X, y, tokenizer, max_sequence_len

# Build the LSTM model
def build_model(vocab_size, max_sequence_len):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, 100, input_length=max_sequence_len - 1),
        tf.keras.layers.LSTM(150),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(vocab_size, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Generate text using the trained model
def generate_text(seed_text, next_words, model, max_sequence_len, tokenizer):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Load the dataset and prepare sequences
file_path = '/content/cleaned_poems.json'
poems_text = read_poems_from_json(file_path)

X, y, tokenizer, max_sequence_len = prepare_sequences(poems_text, max_sequence_len=200)

# Build and train the model
model = build_model(vocab_size=len(tokenizer.word_index) + 1, max_sequence_len=max_sequence_len)
model.fit(X, y, epochs=100, verbose=2)

# Generate new text
seed_text = "The sun"
next_words = 20
generated_text = generate_text(seed_text, next_words, model, max_sequence_len, tokenizer)
print(generated_text)


Epoch 1/100
744/744 - 102s - 136ms/step - accuracy: 0.0588 - loss: 7.1489
Epoch 2/100
744/744 - 99s - 134ms/step - accuracy: 0.0662 - loss: 6.7329
Epoch 3/100
744/744 - 99s - 134ms/step - accuracy: 0.0718 - loss: 6.5086
Epoch 4/100
744/744 - 142s - 190ms/step - accuracy: 0.0804 - loss: 6.2758
Epoch 5/100
744/744 - 142s - 190ms/step - accuracy: 0.0949 - loss: 5.9983
Epoch 6/100
744/744 - 100s - 135ms/step - accuracy: 0.1033 - loss: 5.7282
Epoch 7/100
744/744 - 141s - 190ms/step - accuracy: 0.1138 - loss: 5.4695
Epoch 8/100
744/744 - 141s - 189ms/step - accuracy: 0.1234 - loss: 5.2122
Epoch 9/100
744/744 - 143s - 192ms/step - accuracy: 0.1385 - loss: 4.9630
Epoch 10/100
744/744 - 141s - 190ms/step - accuracy: 0.1554 - loss: 4.7219
Epoch 11/100
744/744 - 142s - 191ms/step - accuracy: 0.1785 - loss: 4.4822
Epoch 12/100
744/744 - 141s - 189ms/step - accuracy: 0.2050 - loss: 4.2495
Epoch 13/100
744/744 - 143s - 193ms/step - accuracy: 0.2318 - loss: 4.0237
Epoch 14/100
744/744 - 142s - 191ms/

In [7]:
# Generate new text
seed_text = "The sun"
next_words = 30
generated_text = generate_text(seed_text, next_words, model, max_sequence_len, tokenizer)
print(generated_text)

The sun went out to work — no the town and pleased it was as she was not be a brook in the finished picture the april spirit through the fac simile


In [8]:
model.save('lstm_poem_model.h5')

In [9]:
import pickle

# Save the tokenizer using pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [10]:
seed_text = "love"
next_words = 50
generated_text = generate_text(seed_text, next_words, model, max_sequence_len, tokenizer)
print(generated_text)

love by memorial mould of light or most in the freckled rose of which writing delinquencies the surroundings in which any of emily dickinson's verses are known to have been written usually serve to explain them clearly but in general the present volume is full of thoughts needing no interpretation to


In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

# Load the saved model
from tensorflow.keras.models import load_model
model = load_model('/content/lstm_poem_model.h5')

# Load the saved tokenizer
with open('/content/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Generate text function
def generate_text(seed_text, next_words, model, max_sequence_len, tokenizer):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Set parameters for generation
seed_text = "THE GRASS"  # The seed word
next_words = 40      # Number of words to generate

# Generate the poem
max_sequence_len = 200  # Set to your original max sequence length
generated_text = generate_text(seed_text, next_words, model, max_sequence_len, tokenizer)
print(generated_text)


THE GRASS so little has to do — its unknown and bees no must have the more volume of a thought has found a response as wide and sympathetic as it has been unexpected even to those who knew best her compelling
